# DATASCI W261: Machine Learning at Scale
## Section 3
## Homework, Week 2
## Name: T.Thomas
## Email: tgthomas@berkeley.edu
## Submission Date:  1/26/2016 5:00AM PST

### nbviewer link:
http://nbviewer.jupyter.org/urls/dl.dropbox.com/s/xqq6kzefz3unn1s/MIDS-W261-2016-Hwk-Week02-Thomas.ipynb

### pdf link:
https://www.dropbox.com/s/bnmgs2tqxrvrd4n/MIDS-W261-2016-Hwk-Week02-Thomas.T.pdf?dl=0

## Week 2 ASSIGNMENTS using Hadoop Streaming and Python

## HW 2.0.  

### What is a race condition in the context of parallel computation? Give an example.

**Parallel Computing:**
Parallel computing is a form of computation in which many calculations are carried out simultaneously. This computational paradigm is often applied to large computational problems that can be divided into smaller disconnected computational tasks wich are then executed concurrently ("in parallel"). Eventually the different individual computations are aggregated/combined to yield results or for additional computations.

However, software written to perform concurrent compuational operations are complicated,due to additional challenges and software bugs arising from having to use shared data and resources. ***Race conditions*** are one such category of bugs that arise when access/modification of shared data and resources are not synchronized.

As an example, consider a typical sequential computational flow as follows. The program clearly prints 3 as expected (folowing the sequential flow)
```python 
    var X = 1 #Shared Variable X    
    def func_A
        X = X + 1    
    def func_B
        X = X + 1
    ...
    ...
    func_A()
    func_B()
    print X    
    #-------prints 3
```    

Now consider a parallel computational scenario where two threads could simultaneously attempt to increment the variable X

```python 
    var X = 1 #Shared Variable X    
    def Thread_A
         X = X + 1
    def Thread_B
         X = X + 1
    ...
    ...
    #Threads A & B are simultaneously called (parallel)
    
    print X  #The output is non-deterministic 
    #-------prints ? 
```    
Some scenarion that can happen when the above code is executed.

* Scenario 1: X = X + 1 
    - A and B could both read the initial value of X and then overwrite each other 
    - Final result: ***X = X + 1 = 2*** 

* Scenario 2: X = X + 2
    - A reads and writes, then B reads and writes
    - Final result: ***X = X + 2 = 3***
    
The value of X above is non-deterministic and this is due to the Race Condition. In the above case, depending on the execution platform, the computer could order the sequence of instructions in different ways leading to different/unpredictable values for X. This is a Race Condition.

### What is MapReduce?

At a very high level MapReduce can be viewed as a programming paradigm that 'codifies' a generic “recipe” for processing large datasets - broken down into two steps or stages. In the   

* ***First stage (map)***, a user-specified computation/transformation is applied in parallel over all input records in a dataset, yielding an intermediate data set,that is then fed into a   

* ***Second stage (fold)*** user-specified computation that may aggregate the output from the stage. 

These two stages are typically referred to as ***map*** and ***fold***, taking roots from the concept of *higher order functions* found in functional programming languages. 

This is depicated in the image below: https://www.dropbox.com/s/r8ri91snatyojry/Mapreduce_Basic.png?dl=0
<img src="Mapreduce_Basic.png" height="500" width="700" border=100>

*Source: Lin, J., & Dyer, C. (2010). Data-intensive text processing with MapReduce. San Rafael, CA: Morgan & Claypool Publishers.* 

We can consider *map* as a concise way to represent the parallel transformation of a dataset (as defined by the function ***$f$***. Similarly *fold* is a concise way to represent an aggregation operation, as defined by the function ***$g$***. A programmer who then is trying to process large datasets, will define these two stages of *map & fold* computations usually within the context of an execution framework which coordinates the actual processing.  

Put another way, MapReduce can refer to three distinct but related concepts.

* First MapReduce can be viewed as a programming model discussed above. 
* Second, MapReduce can be an execution framework that orchestrates  execution of programs written in the map/fold stages. 
* Finally, MapReduce can be viewed as a stack of software implementations of the programming model and the execution framework (eg. Hadoop)


### How does it differ from Hadoop?

MapReduce as discussed above can be a combination of Software implemenation and execution framework that 'codifies' all the software parts required to run the map and reduce/fold stage. Besides just the software functionlity for the Map and Reduce/fold stages, any form of parallel processing implemenation must take into consideration issues like, failure of hardware running the parallel computations (data/code redundancy), moving data efficiently across the connected computing nodes (distributed data storage, high data throughput), ability to scale horizontally as data volume increases etc. A number of Open source implementations have emerged over the past years that implement such fault tolerant, effecient execution frameworks, allowing the developers to just focus on building the map and reduce code. 

Hadoop is one such Open Source implemenation of the MapReduce programming paradigm, complete with a Java-based software implementation and an execution framework that simplifies the building of map and reduce/fold stages in addition to providing fault tolerance, data throughput and scalable processing power. It is part of the Apache project sponsored by the Apache Software Foundation.

### Which programming paradigm is Hadoop based on? Explain and give a simple example in code and show the code running.

Hadoop is based on the MapReduce Programming paradigm (See What is MapReduce ? above)

Hadoop provides a Java-based programming framework that implements MapReduce progamming paradigm. Hadoop does all the work in the back-end required to perform the distributed orchestration among the different compute nodes that will perform the map and reduce tasks, along with distributed data storage with the Hadoop Distributed File System - **HDFS**. Hadoop is configurable , provides fault tolerance and the user just needs to provide code for teh mapper and reducer and Hadoop does the rest.

As an example consider a simple word count problem that can be solved in a distrbuted fashion,  having a mapper program (outputs every word occuring in a text) and a reducer program (aggregates counts for each of the words). The text input, mapper code and reducer code are provided to the hadoop execution framework , which then executes the map and fold/reduce stages and writes the reduced output to the configured output directory. In this example, the user only needs to provide code for the mapper and reducer and the input and output parameters. Hadoop does all the work of splitting the input among multiple mappers and then aggregrating the results with multiple reducers. 

*Note:* Below example runs on a single node cluster.

#### Input Text 

In [65]:
%%writefile wordcount.txt
betty bought some butter 
but the butter betty bought was bitter
so betty bought some better butter to make the bitter batter better 

Overwriting wordcount.txt


#### Mapper

In [66]:
%%writefile hadoop_test_mapper.py
#!/usr/bin/python
import sys
# input comes from STDIN (standard input)
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()
    # split the line into words
    words = line.split()
    # increase counters
    for word in words:
        # write the results to STDOUT (standard output);
        # what we output here will be the input for the
        # Reduce step, i.e. the input for reducer.py
        #
        # tab-delimited; the trivial word count is 1
        print '%s\t%s' % (word, 1)

Overwriting hadoop_test_mapper.py


#### Reducer

In [2]:
%%writefile hadoop_test_reducer.py
#!/usr/bin/python
from operator import itemgetter
import sys

current_word = None
current_count = 0
word = None

# input comes from STDIN
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()

    # parse the input we got from mapper.py
    word, count = line.split('\t', 1)

    # convert count (currently a string) to int
    try:
        count = int(count)
    except ValueError:
        # count was not a number, so silently
        # ignore/discard this line
        continue

    # this IF-switch only works because Hadoop sorts map output
    # by key (here: word) before it is passed to the reducer
    if current_word == word:
        current_count += count
    else:
        if current_word:
            # write result to STDOUT
            print '%s\t%s' % (current_word, current_count)
        current_count = count
        current_word = word

# do not forget to output the last word if needed!
if current_word == word:
    print '%s\t%s' % (current_word, current_count)

Overwriting hadoop_test_reducer.py


#### Test Mapper & Reducer using unix commandline.

In [68]:
!cat wordcount.txt | python hadoop_test_mapper.py | sort -k1,1 | python hadoop_test_reducer.py

batter	1
better	2
betty	3
bitter	2
bought	3
but	1
butter	3
make	1
so	1
some	2
the	2
to	1
was	1


#### Start All Hadoop Services

#### Create folders for output, delete previous results & copy our WordCount.txt to hdfs

In [10]:
#Folder already exists
#!hdfs dfs -mkdir -p /user/dsq

!hdfs dfs -rm /user/dsq/wordcount.txt
!hdfs dfs -rm /user/dsq/wordcountOutput/*
!hdfs dfs -rmdir /user/dsq/wordcountOutput
!hdfs dfs -put wordcount.txt /user/dsq

16/01/25 19:19:00 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/25 19:19:01 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/dsq/wordcount.txt
16/01/25 19:19:03 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/25 19:19:04 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/dsq/wordcountOutput/_SUCCESS
16/01/25 19:19:04 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/dsq/wordcountOutput/part-00000
16/01/25 19:19:04 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/dsq/wordcountOutput/pa

####Run Hadoop Streaming job 

<pre>
hadoop jar hadoop-streaming-2.7.1.jar \
    -D stream.num.map.output.key.fields=n \
    -mapper hadoop_test_mapper.py \
    -reducer hadoop_test_reducer.py \
    -input wordcount.txt \
    -output wordcountOutput</pre>
    

In [11]:
#!$HADOOP_INSTALL/bin/hadoop jar $HADOOP_INSTALL/hadoop-streaming-2.7.1.jar 
    # -mapper hadoop_test_mapper.py 
    # -reducer hadoop_test_reducer.py 
    # -input wordcount.txt -output wordcountOutput 
    # -file hadoop_test_mapper.py 
    # -file hadoop_test_reducer.py 
    # -jobconf mapred.reduce.tasks=1 


!$HADOOP_INSTALL/bin/hadoop jar $HADOOP_INSTALL/hadoop-streaming-2.7.1.jar -mapper hadoop_test_mapper.py -reducer hadoop_test_reducer.py -input wordcount.txt -output wordcountOutput -file hadoop_test_mapper.py -file hadoop_test_reducer.py -jobconf mapred.reduce.tasks=1 

16/01/25 19:20:05 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
16/01/25 19:20:05 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/25 19:20:06 WARN streaming.StreamJob: -jobconf option is deprecated, please use -D instead.
16/01/25 19:20:06 INFO Configuration.deprecation: mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
packageJobJar: [hadoop_test_mapper.py, hadoop_test_reducer.py] [] /tmp/streamjob1500987317932814861.jar tmpDir=null
16/01/25 19:20:06 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/01/25 19:20:06 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/01/25 19:20:06 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/01/25 19:20:07 INFO mapred.FileInputFormat: Total inpu

#### Check results
Check results in our output folder wordcountOutput in hdfs.

In [12]:
!hdfs dfs -cat wordcountOutput/part-00000

16/01/25 19:20:41 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
batter	1
better	2
betty	3
bitter	2
bought	3
but	1
butter	3
make	1
so	1
some	2
the	2
to	1
was	1


## HW 2.1. Sort in Hadoop MapReduce

Given as input: Records of the form <integer, “NA”>, where integer is any integer, and “NA” is just the empty string.
Output: sorted key value pairs of the form <integer, “NA”> in decreasing order; 

* What happens if you have multiple reducers? Do you need additional steps? Explain.
* Write code to generate N  random records of the form <integer, “NA”>. Let N = 10,000.
* Write the python Hadoop streaming map-reduce job to perform this sort. 
* Display the top 10 biggest numbers. Display the 10 smallest numbers


#### Generate sortinput.txt with 10,000 random <integer,"NA"> pairs 

In [13]:
%%writefile sortinputgen.py
#!/usr/bin/python
## mapper.py
## Author: Tigi Thomas
## Description: mapper code for HW1.2-1.5
import numpy as np

sort_raw = []
for i in range(1,10001):
    sort_raw.append([i,"NA"])
    
X = np.array(sort_raw)
np.random.seed(0)
shuffle = np.random.permutation(np.arange(X.shape[0]))
X = X[shuffle]

for i in xrange(X.shape[0]):
    print X[i][0] + '\t' + X[i][1]  # is your pair

Overwriting sortinputgen.py


#### Generate the sortinput.txt. This will be passed to hadoop map/reduce

In [14]:
!python sortinputgen.py > sortinput.txt

In [15]:
%%writefile hadoop_sort_mapper.py
#!/usr/bin/python
from operator import itemgetter
import sys

current_word = None
current_count = 0
word = None

# input comes from STDIN
for line in sys.stdin:
    # remove leading and trailing whitespace
    print line.strip()

Overwriting hadoop_sort_mapper.py


In [16]:
%%writefile hadoop_sort_reducer.py
#!/usr/bin/python
from operator import itemgetter
import sys

current_word = None
current_count = 0
word = None

# input comes from STDIN
for line in sys.stdin:
    # remove leading and trailing whitespace
    print line.strip()

Overwriting hadoop_sort_reducer.py


#### Test 

In [17]:
#Generete the sorted output just with the reducer
!cat sortinput.txt | python hadoop_sort_mapper.py | sort -g -r | python hadoop_sort_reducer.py > sortedout.txt 

In [18]:
#see the top 10
!cat sortedout.txt | head -10

10000	NA
9999	NA
9998	NA
9997	NA
9996	NA
9995	NA
9994	NA
9993	NA
9992	NA
9991	NA
cat: write error: Broken pipe


In [19]:
#!hdfs dfs -mkdir -p /user/dsq

In [20]:
!hdfs dfs -rm /user/dsq/sortinput.txt
!hdfs dfs -rm /user/dsq/sortoutput/*
!hdfs dfs -rmdir /user/dsq/sortoutput
!hdfs dfs -put sortinput.txt /user/dsq

16/01/25 19:21:30 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/25 19:21:31 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/dsq/sortinput.txt
16/01/25 19:21:33 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/25 19:21:34 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/dsq/sortoutput/_SUCCESS
16/01/25 19:21:34 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/dsq/sortoutput/part-00000
16/01/25 19:21:36 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/25 19:21:38 WARN util.NativeC

In [21]:
# Run the Hadoop Streaming Job
#!$HADOOP_INSTALL/bin/hadoop jar $HADOOP_INSTALL/hadoop-streaming-2.7.1.jar 
#    -D mapred.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator 
#    -D mapred.text.key.comparator.options='-n -r' 
#    -mapper hadoop_sort_mapper.py 
#    -reducer hadoop_sort_reducer.py 
#    -input sortinput.txt 
#    -output sortoutput 
#    -file sortinput.txt 
#    -file hadoop_sort_mapper.py 
#    -file hadoop_sort_reducer.py 
#    -jobconf mapred.reduce.tasks=1

!$HADOOP_INSTALL/bin/hadoop jar $HADOOP_INSTALL/hadoop-streaming-2.7.1.jar -D mapred.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator -D mapred.text.key.comparator.options='-n -r' -mapper hadoop_sort_mapper.py -reducer hadoop_sort_reducer.py -input sortinput.txt -output sortoutput -file sortinput.txt -file hadoop_sort_mapper.py -file hadoop_sort_reducer.py -jobconf mapred.reduce.tasks=1   

16/01/25 19:23:53 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
16/01/25 19:23:53 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/25 19:23:54 WARN streaming.StreamJob: -jobconf option is deprecated, please use -D instead.
16/01/25 19:23:54 INFO Configuration.deprecation: mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
packageJobJar: [sortinput.txt, hadoop_sort_mapper.py, hadoop_sort_reducer.py] [] /tmp/streamjob4947641602306020312.jar tmpDir=null
......16/01/25 19:23:56 INFO mapred.LocalDistributedCacheManager: Localized file:/home/dsq/Dropbox/DataScienceBerkeley/Semester4/W261/Week2_MapReduce/hadoop_sort_mapper.py as file:/app/hadoop/tmp/mapred/local/1453778636083/hadoop_sort_mapper.py
16/01/25 19:23:56 INFO mapred.LocalDistributedCacheManager: Localized file:/home/dsq/Dropbox/DataScienceBerkeley/Semester4/W261/Week2_MapReduce/

#### Display the top 10 biggest numbers

**NOTE**
The reducer was configured to sort in reverse numerical key, so ***head -10*** will show the top 10 biggest numbers

In [22]:
!hdfs dfs -cat sortoutput/part-00000 | head -10

16/01/25 19:24:14 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
10000	NA
9999	NA
9998	NA
9997	NA
9996	NA
9995	NA
9994	NA
9993	NA
9992	NA
9991	NA
cat: Unable to write to output stream.


#### Display the 10 smallest numbers

**NOTE**
The reducer was configured to sort in reverse numerical key, so ***tail -10*** will show the 10 smallest numbers

In [24]:
!hdfs dfs -cat sortoutput/part-00000 | tail -10

16/01/25 19:24:55 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
10	NA
9	NA
8	NA
7	NA
6	NA
5	NA
4	NA
3	NA
2	NA
1	NA


#### What happens if you have multiple reducers? Do you need additional steps? Explain.

Intermediate data is sorted by key and passed onto the Reducers. If there are multiple reducers, then no ordering relationship is guaranteed for keys across the different reducers. Hence if we do have multiple reducers, additional steps might be required to combine the sorted results output from each reducer.

## HW 2.2.  WORDCOUNT
Using the Enron data from HW1 and Hadoop MapReduce streaming, write the mapper/reducer job that  will determine the word count (number of occurrences) of each white-space delimitted token (assume spaces, fullstops, comma as delimiters). Examine the word “assistance” and report its word count results.

CROSSCHECK: >grep assistance enronemail_1h.txt|cut -d$'\t' -f4| grep assistance|wc -l    
8    
*NOTE*  "assistance" occurs on 8 lines but how many times does the token occur? 10 times! This is the number we are looking for!

#### Mapper

**Note**: Filtering out some stop words

In [5]:
%%writefile hadoop_wordcount_mapper.py
#!/usr/bin/python
## mapper.py
## Author: Tigi Thomas
## Description: mapper code for HW1.2-1.5
import sys
import re

#fixed list of stop words
stopwords = ['a','able','about','across','after','all','almost','also','am','among',
             'an','and','any','are','as','at','be','because','been','but','by','can',
             'cannot','could','dear','did','do','does','either','else','ever','every',
             'for','from','get','got','had','has','have','he','her','hers','him','his',
             'how','however','i','if','in','into','is','it','its','just','least','let',
             'like','likely','may','me','might','most','must','my','neither','no','nor',
             'not','of','off','often','on','only','or','other','our','own','rather','said',
             'say','says','she','should','since','so','some','than','that','the','their',
             'them','then','there','these','they','this','tis','to','too','twas','us',
             'wants','was','we','were','what','when','where','which','while','who',
             'whom','why','will','with','would','yet','you','your']

#Use some pre-compiled re-gex for punctuation and numbers (exclude period and comma)
punctpattern = re.compile(r'[\.\^\$\*\+\-\=\{\}\[\]\\\|\(\)<>-@&#%_=!?:;,/\'\"]')
nonprintable = re.compile('[^\s!-~]')
#since we are going to consider period and comma as delimitter in addition to space

#take out numbers..
numpatt1 = re.compile(r'\b[0-9]{1,100}\b')
numpatt2 = re.compile(r'[0-9]{1,100}\b')
numpatt3 = re.compile(r'\b[0-9]{1,100}')

# Preprocess any text to remove punctuations, numbers, convert to lower etc.
def preprocess_txt(s): 
    s = s.lower()
    s = re.sub(nonprintable, r' ', s)
    s = re.sub(punctpattern, r' ', s)
    s = re.sub(numpatt1, r'', s )
    s = re.sub(numpatt2, r'', s )
    s = re.sub(numpatt3, r'', s )
    return s


# input comes from STDIN (standard input)
for line in sys.stdin:
    
    # remove leading and trailing whitespace
    line = line.strip()
    
    # pre-process the line
    line = preprocess_txt(line)
    
    # split the line into words
    words = line.split()
    
    #filter out the stop words
    filtered_words = [tk for tk in words if tk not in stopwords]
    
    # increase counters
    for word in filtered_words:
        # write the results to STDOUT 
        # tab-delimited; the trivial word count is 1
        print word + '\t' + '1'

Overwriting hadoop_wordcount_mapper.py


#### Reducer 

In [6]:
%%writefile hadoop_wordcount_reducer.py
#!/usr/bin/python
from operator import itemgetter
import sys

sys.stderr.write("reporter:counter:HowManyReducers,Reducer,1\n")

current_word = None
current_count = 0
word = None

# input comes from STDIN
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()

    # parse the input we got from mapper.py
    word, count = line.split('\t', 1)

    # convert count (currently a string) to int
    try:
        count = int(count)
    except ValueError:
        # count was not a number, so silently
        # ignore/discard this line
        continue

    # this IF-switch only works because Hadoop sorts map output
    # by key (here: word) before it is passed to the reducer
    if current_word == word:
        current_count += count
    else:
        if current_word:
            # write result to STDOUT
            print '%s\t%s' % (current_word, current_count)
        current_count = count
        current_word = word

# do not forget to output the last word if needed!
if current_word == word:
    print '%s\t%s' % (current_word, current_count)

Overwriting hadoop_wordcount_reducer.py


In [7]:
# Clear the old input data and output data
!hdfs dfs -rm /user/dsq/enronemail_1h.txt
!hdfs dfs -rm /user/dsq/onlywordcountoutput/*
!hdfs dfs -rmdir /user/dsq/onlywordcountoutput
!hdfs dfs -put enronemail_1h.txt /user/dsq



rm: `/user/dsq/enronemail_1h.txt': No such file or directory
rm: `/user/dsq/onlywordcountoutput/_temporary': Is a directory
rmdir: `/user/dsq/onlywordcountoutput': Directory is not empty


In [10]:
#!$HADOOP_INSTALL/bin/hadoop jar $HADOOP_INSTALL/hadoop-streaming-2.7.1.jar 
    # -mapper hadoop_wordcount_mapper.py 
    # -reducer hadoop_wordcount_reducer.py 
    # -input enronemail_1h.txt 
    # -output onlywordcountoutput 
    # -file hadoop_wordcount_mapper.py 
    # -file hadoop_wordcount_reducer.py 
    # -jobconf mapred.reduce.tasks=1 

!$HADOOP_INSTALL/bin/hadoop jar $HADOOP_INSTALL/hadoop-streaming-2.7.1.jar -mapper hadoop_wordcount_mapper.py -reducer hadoop_wordcount_reducer.py -input enronemail_1h.txt -output onlywordcountoutput -file hadoop_wordcount_mapper.py -file hadoop_wordcount_reducer.py -jobconf mapred.reduce.tasks=1 

16/01/26 00:04:10 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
16/01/26 00:04:10 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/26 00:04:11 WARN streaming.StreamJob: -jobconf option is deprecated, please use -D instead.
		...		...	Map-Reduce Framework
		Map input records=100
		Map output records=19476
		Map output bytes=173009
		Map output materialized bytes=211967
		Input split bytes=101
		Combine input records=0
		Combine output records=0
		Reduce input groups=4955
		Reduce shuffle bytes=211967
		Reduce input records=19476
		Reduce output records=4955
		Spilled Records=38952
		Shuffled Maps =1
		Failed Shuffles=0
		Merged Map outputs=1
		GC time elapsed (ms)=52
		Total committed heap usage (bytes)=479199232
		...		...16/01/26 00:04:21 INFO streaming.StreamJob: Output directory: onlywordcountoutput


#### Examine the word “assistance” and report its word count results.

In [11]:
#Check Count of "assistance"
!hdfs dfs -cat onlywordcountoutput/part-00000 | grep assistance

16/01/26 00:06:39 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
assistance	10


## HW 2.2.1  Using Hadoop MapReduce and your wordcount job (from HW2.2) determine the top-10 occurring tokens (most frequent tokens)

***NOTE***

To accomplish this 

* we can take the output from H2.2 'onlywordcountoutput/part-00000 and pass it thru a mapper that just reverses (word, count) to (count, word). 
* Use the hadoop_sort_reducer.py from HW2.1 to get it sorted

In [14]:
%%writefile hadoop_wordcount_reverse_mapper.py
#!/usr/bin/python
import sys
# input comes from STDIN (standard input)
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()
    
    # split the line into words
    word, count = line.split('\t',1)
    
    # We just reverse it as Count and Word
    # This will be fed to reducer which we can configure 
    # to sort numerically
    print '%s\t%s' % (count, word)

Writing hadoop_wordcount_reverse_mapper.py


In [16]:
#Clear any existing outputs.. we leave the input from HW2.2 intact
!hdfs dfs -rm /user/dsq/reversewordcountoutput/*
!hdfs dfs -rmdir /user/dsq/reversewordcountoutput

16/01/26 00:26:27 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
rm: `/user/dsq/reversewordcountoutput/*': No such file or directory
16/01/26 00:26:29 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
rmdir: `/user/dsq/reversewordcountoutput': No such file or directory


In [17]:
#!$HADOOP_INSTALL/bin/hadoop jar $HADOOP_INSTALL/hadoop-streaming-2.7.1.jar 
    # -D mapred.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator 
    # -D mapred.text.key.comparator.options='-n -r' 
    # -mapper hadoop_wordcount_reverse_mapper.py 
    # -reducer hadoop_sort_reducer.py 
    # -input onlywordcountoutput/part-00000 
    # -output reversewordcountoutput 
    # -file hadoop_wordcount_reverse_mapper.py 
    # -file hadoop_sort_reducer.py 
    # -jobconf mapred.reduce.tasks=1 

!$HADOOP_INSTALL/bin/hadoop jar $HADOOP_INSTALL/hadoop-streaming-2.7.1.jar -D mapred.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator -D mapred.text.key.comparator.options='-n -r' -mapper hadoop_wordcount_reverse_mapper.py -reducer hadoop_sort_reducer.py -input onlywordcountoutput/part-00000 -output reversewordcountoutput -file hadoop_wordcount_reverse_mapper.py -file hadoop_sort_reducer.py -jobconf mapred.reduce.tasks=1 

16/01/26 00:26:43 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
16/01/26 00:26:44 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
		...		...	Map-Reduce Framework
		Map input records=4955
		Map output records=4955
		Map output bytes=48863
		Map output materialized bytes=58779
		Input split bytes=114
		Combine input records=0
		Combine output records=0
		Reduce input groups=67
		Reduce shuffle bytes=58779
		Reduce input records=4955
		Reduce output records=4955
		Spilled Records=9910
		Shuffled Maps =1
		Failed Shuffles=0
		Merged Map outputs=1
		GC time elapsed (ms)=101
		Total committed heap usage (bytes)=477626368
		...		...16/01/26 00:26:52 INFO streaming.StreamJob: Output directory: reversewordcountoutput


#### Determine the top-10 occurring tokens (most frequent tokens)

In [18]:
!hdfs dfs -cat reversewordcountoutput/part-00000 | head -10

16/01/26 00:28:16 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
382	ect
227	com
206	hou
149	s
127	enron
86	free
84	please
76	~
71	information
70	out
cat: Unable to write to output stream.


## HW 2.3. Multinomial NAIVE BAYES with NO Smoothing 

Using the Enron data from HW1 and Hadoop MapReduce, write  a mapper/reducer job(s) that will both learn  Naive Bayes classifier and classify the Enron email messages using the learnt Naive Bayes classifier. Use all white-space delimitted tokens as independent input variables (assume spaces, fullstops, commas as delimiters). Note: for multinomial Naive Bayes, the Pr(X=“assistance”|Y=SPAM) is calculated as follows:

the number of times “assistance” occurs in SPAM labeled documents / the number of words in documents labeled SPAM 

E.g.,   “assistance” occurs 5 times in all of the documents Labeled SPAM, and the length in terms of the number of words in all documents labeled as SPAM (when concatenated) is 1,000. Then Pr(X=“assistance”|Y=SPAM) = 5/1000. Note this is a multinomial estimation of the class conditional for a Naive Bayes Classifier. No smoothing is needed in this HW. Multiplying lots of probabilities, which are between 0 and 1, can result in floating-point underflow. 
Since log(xy) = log(x) + log(y), it is better to perform all computations by summing logs of probabilities rather than multiplying probabilities. Please pay attention to probabilites that are zero! They will need special attention. 

**Count up how many times you need to process a zero probabilty for each class and report.**

**Report the performance of your learnt classifier in terms of misclassifcation error rate of your multinomial Naive Bayes Classifier. Plot a histogram of the  posterior probabilities (i.e., Pr(Class|Doc)) for each class over the training set. Summarize what you see.**

*Error Rate* = misclassification rate with respect to a provided set (say training set in this case). It is more formally defined here:

Let DF represent the evalution set in the following:
Err(Model, DF) = |{(X, c(X)) ∈ DF : c(X) != Model(x)}|   / |DF|

Where || denotes set cardinality; c(X) denotes the class of the tuple X in DF; and Model(X) denotes the class inferred by the Model “Model”

#### Mapper code for NB Classification

In [25]:
%%writefile mapper.py
#!/usr/bin/python
## mapper.py
## Author: Tigi Thomas
## Description: mapper code for HW2.3 - 2.5
import sys
import re

#fixed list of stop words
stopwords = ['a','able','about','across','after','all','almost','also','am','among',
             'an','and','any','are','as','at','be','because','been','but','by','can',
             'cannot','could','dear','did','do','does','either','else','ever','every',
             'for','from','get','got','had','has','have','he','her','hers','him','his',
             'how','however','i','if','in','into','is','it','its','just','least','let',
             'like','likely','may','me','might','most','must','my','neither','no','nor',
             'not','of','off','often','on','only','or','other','our','own','rather','said',
             'say','says','she','should','since','so','some','than','that','the','their',
             'them','then','there','these','they','this','tis','to','too','twas','us',
             'wants','was','we','were','what','when','where','which','while','who',
             'whom','why','will','with','would','yet','you','your']

#Use some pre-compiled re-gex for punctuation and numbers (exclude period and comma)
punctpattern = re.compile(r'[\.\^\$\*\+\-\=\{\}\[\]\\\|\(\)<>-@&#%_=!?:;,/\'\"]')
nonprintable = re.compile('[^\s!-~]')
#since we are going to consider period and comma as delimitter in addition to space

#take out numbers..
numpatt1 = re.compile(r'\b[0-9]{1,100}\b')
numpatt2 = re.compile(r'[0-9]{1,100}\b')
numpatt3 = re.compile(r'\b[0-9]{1,100}')

# Preprocess any text to remove punctuations, numbers, convert to lower etc.
def preprocess_txt(s): 
    s = s.lower()
    s = re.sub(nonprintable, r' ', s)
    s = re.sub(punctpattern, r' ', s)
    s = re.sub(numpatt1, r'', s )
    s = re.sub(numpatt2, r'', s )
    s = re.sub(numpatt3, r'', s )
    return s

#initialize arguments
count = 0

# input comes from STDIN (standard input)
for line in sys.stdin:
    
        # remove leading and trailing whitespace - some clean-up
        line = line.strip().lower()
        emaildata = line.split("\t")
        emaildatalen = len(emaildata)
       
        #extrat the email parts.. 
        #don't really care about the first item, 
        #start from spam or not.. 
        if(emaildatalen >= 2):            
            if(emaildatalen >= 3 ):
                subject = emaildata[2]
            else:
                subject = " "
            
            if(emaildatalen == 4 ):
                body = emaildata[3]
            else:
                body = " "
            
            #Get subject and body of email together
            emailcontent = subject + " "  + body 
            
            #pre-process to remove punctuation etc.
            emailcontent = preprocess_txt(emailcontent)
            
            #extract words now and filter out stop-words
            words = emailcontent.split()
            filtered_words = [tk for tk in words if tk not in stopwords]

            #we write the ID , current Spam classification, total word count in document
            #the output file will contain one row for every document
            if emaildata[1] == "1":
                outtxt = emaildata[0]+'\t'+ "1" +'\t'+ str(len(filtered_words)) 
            else:
                outtxt = emaildata[0]+'\t'+ "0" +'\t'+ str(len(filtered_words)) 
    
            #Prepare our word count list
            #Add word to dict if in findword list or all words '*' specified
            wc = {}
            for word in filtered_words:
                    wc[word] = wc.get(word, 0) + 1
            
            #Stick the occurence of word=count for every word in document
            #for faster processing on the reducer side.
            wordcounttxt = "\t*"
            if(len(wc) > 0):
                wordcounttxt = "";
                for word, count in wc.iteritems():
                    wordcounttxt += '\t'+word+"="+str(count)
            
            #write out document row with 
            print outtxt + wordcounttxt

Overwriting mapper.py


In [26]:
!chmod a+x mapper.py

#### Reducer Code. 

In [20]:
%%writefile reducer.py
#!/usr/bin/python
## reducer.py
## Author: T.Thomas
## Description: reducer code for HW1.2-1.5
import sys
import math


def printcollection(Title, coll):
    if(Title != ""):
        print ""
        print Title          
    for k, v in coll.iteritems():
        print k, v


# some dict structures to keep track of intermediate data
wc = {}
spwc = {}
nb = {}
w_condprob = {}
testdata = {}

#Change here to control smoothing...
#lp_smooth = False  #--For HW 2.3
lp_smooth = True #--For HW 2.4, 2.5

# input comes from STDIN
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()
    item = line.split("\t")

    #extract total spam=1 and spam=0 doc/email counts
    nb["spam"+item[1]+"dc"] = nb.get("spam"+item[1]+"dc", 0) + 1
    #extract spam=1 and spam=0 word counts
    nb["spam"+item[1]+"wc"] = nb.get("spam"+item[1]+"wc", 0) + eval(item[2])

    #extract the words and their counts per document
    #this way we have how many times word occurs in document
    #and how many times it occurs in a spam document
    if(item[3].strip() != "*"):
        for itm in item[3:]:
            word, count = itm.strip().split("=",1)
            wc[word] = wc.get(word, 0) + eval(count)
            if(item[1] == "1"):                  
                spwc[word] = spwc.get(word, 0) + eval(count)
            #nb[item[0]+"_InSpamCount"] = nb.get(item[0]+"_InSpamCount", 0) + eval(item[2])
    
    #storing test data later for classification. 
    testdata[item[0]] = item[1:]

#Calculate prior probabilities - these will be used everywhere.            
prior_spam =  ( nb["spam1dc"] / float( nb["spam1dc"] + nb["spam0dc"] ) ) 
prior_notspam = 1 - prior_spam 

#setup smoothing parameters if smoothing was specified. see line #58-59 in pNaiveBayes.sh
lp_num = 0
lp_denom = 0   
if (lp_smooth):
    lp_num = 1
    lp_denom = len(wc) 
    
#Here we pre-compute the conditional probablitly for each word
#P(word in email | spam) &  #P(word in email | not spam)
for word, count in wc.iteritems():
    in_spam_count = spwc.get(word,0)
    not_in_spam_count = (count - in_spam_count)
    spam_wordcount = nb.get("spam1wc")
    not_spam_wordcount = nb.get("spam0wc")
    
    spam_condprob =  (in_spam_count + lp_num)/ float(spam_wordcount + lp_num)
    notspam_condprob = (not_in_spam_count + lp_num) / float(not_spam_wordcount + lp_num)
    
    # ----------- Debug check if we have any prob that are 0.0 -----------------
    # if(spam_condprob == 0.0 or notspam_condprob == 0.0):
    #   print word, count, in_spam_count,  not_in_spam_count   
    
    #Store the Conditional spam and notspam prob for every word.
    w_condprob[word] = [spam_condprob, notspam_condprob]
          
#print all words and the count
#printcollection("-----------Total Word Count-----------------", wc)

#print in-spam count for each word
#printcollection("-----------In Spam Word Count-----------------", spwc)

#print some NB model parameters
#printcollection("-----------Counts for Naive Bayes Model -------------", nb)

#print computed probabilities after running thru training set
#print ""
#print "------Computed Probabilities from Training Set---"
#print "P(prior_spam) = {0:.5f}".format(prior_spam)
#print "P(prior_not_Spam) = {0:.5f}".format(prior_notspam)

#print ""
#print "Word [ P(spam | word in email), P(not spam | word in email) ]"
#print "-------------------------------------------------------------"
#printcollection("", w_condprob)


#print classification - classify test data set
#print ""
#print "RESULTS: Classification of Test Data ----------"
#print ""
#print "ID  Truth(Spam/Ham : 1/0) Class(Spam/Ham : 1/0)"
#print "-----------------------------------------------"
result = {}

#run the model on our data set classifying all emails.
#note: 
#the first item in w_condprob, namely w_condprob[word][0] stores P(word in email | spam)
#the second item in w_condprob, namely w_condprob[word][1] stores P(word in email | not spam)
doc_count = len(testdata)
error_count = 0
zeroprobcount = 0
for docid, data in testdata.iteritems():
    #data[0] is current classification
    #data[1] is total word count in the document
    #data[2:] contains all the individual words and their counts
    #         and for each word we compute P(spam | word in email ) and P(not spam | word in email)
    #         
    p_spam = math.log(prior_spam)
    p_notspam = math.log(prior_notspam)
    if(data[2].strip() != "*"):
        zeroprobcount = 0
        for itm in data[2:]:            
            word, count = itm.strip().split("=",1)
            if(w_condprob[word][0] > 0.0 and w_condprob[word][1] > 0.0):
                p_spam += math.log(w_condprob[word][0])*eval(count)
                p_notspam += math.log(w_condprob[word][1])*eval(count)
            else:
                zeroprobcount += 1
                
    true_spam_or_ham = data[0]
    classified_spam_or_ham = "0"
    if(p_spam > p_notspam): 
        classified_spam_or_ham = "1"
    
    #Print our classification....
    print docid + '\t' + true_spam_or_ham + '\t' + classified_spam_or_ham + '\t' + str(zeroprobcount) + '\t' + str(p_spam) +'\t' + str(p_notspam) 
    
    if(true_spam_or_ham != classified_spam_or_ham):
        error_count += 1
            
#print ""
#print "Model Error Rate = {0:.5f}".format(error_count/float(doc_count))

#print 

Overwriting reducer.py


In [21]:
!chmod a+x reducer.py

#### Clean any existing outputs and Run Streaming MapReduce Job

In [27]:
!hdfs dfs -rm /user/dsq/HW23output/*
!hdfs dfs -rmdir /user/dsq/HW23output

16/01/26 04:09:39 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
rm: `/user/dsq/HW23output/*': No such file or directory
16/01/26 04:09:41 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
rmdir: `/user/dsq/HW23output': No such file or directory


In [28]:
#!$HADOOP_INSTALL/bin/hadoop jar $HADOOP_INSTALL/hadoop-streaming-2.7.1.jar 
    # -mapper mapper.py 
    # -reducer reducer.py 
    # -input enronemail_1h.txt 
    # -output HW23output 
    # -file mapper.py 
    # -file reducer.py 
    # -file enronemail_1h.txt 
    # -jobconf mapred.reduce.tasks=1

!$HADOOP_INSTALL/bin/hadoop jar $HADOOP_INSTALL/hadoop-streaming-2.7.1.jar -mapper mapper.py -reducer reducer.py -input enronemail_1h.txt -output HW23output -file mapper.py -file reducer.py -file enronemail_1h.txt -jobconf mapred.reduce.tasks=1

16/01/26 04:09:46 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
16/01/26 04:09:46 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/26 04:09:46 WARN streaming.StreamJob: -jobconf option is deprecated, please use -D instead.
16/01/26 04:09:46 INFO Configuration.deprecation: mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
packageJobJar: [mapper.py, reducer.py, enronemail_1h.txt] [] /tmp/streamjob1484723572566919927.jar tmpDir=null
		...
		...
	Map-Reduce Framework
		Map input records=100
		Map output records=100
		Map output bytes=110369
		Map output materialized bytes=110752
		Input split bytes=101
		Combine input records=0
		Combine output records=0
		Reduce input groups=100
		Reduce shuffle bytes=110752
		Reduce input records=100
		Reduce output records=100
		Spilled Records=200
		Shuffled Maps =1
		Failed Shuffles=0
		Merged Map

#### Check output

Copy output to local directory for further analysis, graphing etc.

In [29]:
!rm HW23_Classification_Result.txt
!hdfs dfs -copyToLocal HW23output/part-00000 HW23_Classification_Result.txt

16/01/26 04:10:07 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/26 04:10:09 WARN hdfs.DFSClient: DFSInputStream has been closed already


#### Re-using Training Error Function provided in master solution from HW1

#### Run Training Error for HW 2.3

In [30]:
#Training Error Function

from __future__ import division

def calculate_training_error(pred, true):
    """Calculates the training error given a vector 
    of predictions and a vector of true classes"""
    
    num_wrong=0
    for i in zip(pred,true):
        if i[0]!=i[1]: #If predicted value doesn't equal true value, increment our count
            num_wrong+=1
            
    #Divide number of incorrect examples by total number of examples in the data
    print "Training error: "+str(num_wrong/float(len(pred)))
    
#HW 1.3 Evaluation Code
# To install pandas: at the SHELL prompt type conda install pandas 
import pandas as pd #Use pandas to quickly read results from our output file
#conda install pandas 
def eval_2_3():
    with open('HW23_Classification_Result.txt','rb') as f:
        mr_data=pd.read_csv(f, sep='\t', header=None)
    print "HW 2.3 Multinomial NB Results via Hadoop MapReduce Implementation NO LP Smoothing"
    print "Misclassification Error Rate " 
    calculate_training_error(mr_data[1],mr_data[2])

eval_2_3()

HW 2.3 Multinomial NB Results via Hadoop MapReduce Implementation NO LP Smoothing
Misclassification Error Rate 
Training error: 0.1


## HW 2.4 Repeat HW 2.3 with the following modification: use Laplace plus-one smoothing. Compare the misclassifcation error rates for 2.3 versus 2.4 and explain the differences.

***NOTE***

* Using the same code from 2.3, just setting the LP Smoothing flag to **True** to turn on LP Smoothing.
* Run MapReduce and capture output to new Folder for HW24


In [71]:
!hdfs dfs -rm /user/dsq/HW24output/*
!hdfs dfs -rmdir /user/dsq/HW24output

16/01/26 02:30:05 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/26 02:30:07 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/dsq/HW24output/_SUCCESS
16/01/26 02:30:07 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/dsq/HW24output/part-00000
16/01/26 02:30:08 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [72]:
# !$HADOOP_INSTALL/bin/hadoop jar $HADOOP_INSTALL/hadoop-streaming-2.7.1.jar 
    # -mapper mapper.py 
    # -reducer reducer.py 
    # -input enronemail_1h.txt 
    # -output HW24output 
    # -file mapper.py 
    # -file reducer.py 
    # -file enronemail_1h.txt 
    # -jobconf mapred.reduce.tasks=1

!$HADOOP_INSTALL/bin/hadoop jar $HADOOP_INSTALL/hadoop-streaming-2.7.1.jar -mapper mapper.py -reducer reducer.py -input enronemail_1h.txt -output HW24output -file mapper.py -file reducer.py -file enronemail_1h.txt -jobconf mapred.reduce.tasks=1

16/01/26 02:30:18 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
16/01/26 02:30:19 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/26 02:30:19 WARN streaming.StreamJob: -jobconf option is deprecated, please use -D instead.
16/01/26 02:30:19 INFO Configuration.deprecation: mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
packageJobJar: [mapper.py, reducer.py, enronemail_1h.txt] [] /tmp/streamjob1124503106935049572.jar tmpDir=null
		...
		...
	Map-Reduce Framework
		Map input records=100
		Map output records=100
		Map output bytes=110369
		Map output materialized bytes=110752
		Input split bytes=101
		Combine input records=0
		Combine output records=0
		Reduce input groups=100
		Reduce shuffle bytes=110752
		Reduce input records=100
		Reduce output records=100
		Spilled Records=200
		Shuffled Maps =1
		Failed Shuffles=0
		Merged Map

In [73]:
!rm HW24_Classification_With_LPSmoothing_Result.txt
!hdfs dfs -copyToLocal HW24output/part-00000 HW24_Classification_With_LPSmoothing_Result.txt

16/01/26 02:30:36 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/26 02:30:37 WARN hdfs.DFSClient: DFSInputStream has been closed already


#### Run Training evaluation for HW 2.4

In [15]:
#conda install pandas
import pandas as pd 

def eval_2_4():
    with open('HW24_Classification_With_LPSmoothing_Result.txt','rb') as f:
        mr_data=pd.read_csv(f, sep='\t', header=None)
    print "HW 2.4 Multinomial NB Results via Hadoop MapReduce Implementation With LP Smoothing"
    print "Misclassification Error Rate" 
    calculate_training_error(mr_data[1],mr_data[2])

eval_2_4()

HW 2.4 Multinomial NB Results via Hadoop MapReduce Implementation With LP Smoothing
Misclassification Error Rate
Training error: 0.0


#### Compare the misclassifcation error rates for 2.3 versus 2.4 and explain the differences.

With LP Smoothing the Mis-classification error rate was 0% versus about 10% without smoothing. The difference mainly arises due to words that are missing in the training set that actually occur in the test data. Without any smoothing, we don't assign any probability mass to those words that are missing. Just because we did not see a word in the training set, it does not mean that that it never show up in the real test documents. By Appling smoothing we are providing some minimum probability for every word.

## HW 2.5. Repeat HW 2.4. This time when modeling and classification ignore tokens with a frequency of less than three (3) in the training set. How does it affect the misclassifcation error of learnt naive multinomial Bayesian Classifier on the training dataset:


#### Here we write a new mapper that ignores words with freq < 3.  Then run using previous reducer (with smoothing from HW 2.4)

In [8]:
%%writefile highfreqwords_mapper.py
#!/usr/bin/python
## mapper.py
## Author: Tigi Thomas
## Description: mapper code for HW2.3 - 2.5
import sys
import re

#fixed list of stop words
stopwords = ['a','able','about','across','after','all','almost','also','am','among',
             'an','and','any','are','as','at','be','because','been','but','by','can',
             'cannot','could','dear','did','do','does','either','else','ever','every',
             'for','from','get','got','had','has','have','he','her','hers','him','his',
             'how','however','i','if','in','into','is','it','its','just','least','let',
             'like','likely','may','me','might','most','must','my','neither','no','nor',
             'not','of','off','often','on','only','or','other','our','own','rather','said',
             'say','says','she','should','since','so','some','than','that','the','their',
             'them','then','there','these','they','this','tis','to','too','twas','us',
             'wants','was','we','were','what','when','where','which','while','who',
             'whom','why','will','with','would','yet','you','your']

#Use some pre-compiled re-gex for punctuation and numbers (exclude period and comma)
punctpattern = re.compile(r'[\.\^\$\*\+\-\=\{\}\[\]\\\|\(\)<>-@&#%_=!?:;,/\'\"]')
nonprintable = re.compile('[^\s!-~]')
#since we are going to consider period and comma as delimitter in addition to space

#take out numbers..
numpatt1 = re.compile(r'\b[0-9]{1,100}\b')
numpatt2 = re.compile(r'[0-9]{1,100}\b')
numpatt3 = re.compile(r'\b[0-9]{1,100}')

# Preprocess any text to remove punctuations, numbers, convert to lower etc.
def preprocess_txt(s): 
    s = s.lower()
    s = re.sub(nonprintable, r' ', s)
    s = re.sub(punctpattern, r' ', s)
    s = re.sub(numpatt1, r'', s )
    s = re.sub(numpatt2, r'', s )
    s = re.sub(numpatt3, r'', s )
    return s

#initialize arguments
count = 0

# input comes from STDIN (standard input)
for line in sys.stdin:
    
        # remove leading and trailing whitespace - some clean-up
        line = line.strip().lower()
        emaildata = line.split("\t")
        emaildatalen = len(emaildata)
       
        #extrat the email parts.. 
        #don't really care about the first item, 
        #start from spam or not.. 
        if(emaildatalen >= 2):            
            if(emaildatalen >= 3 ):
                subject = emaildata[2]
            else:
                subject = " "
            
            if(emaildatalen == 4 ):
                body = emaildata[3]
            else:
                body = " "
            
            #Get subject and body of email together
            emailcontent = subject + " "  + body 
            
            #pre-process to remove punctuation etc.
            emailcontent = preprocess_txt(emailcontent)
            
            #extract words now and filter out stop-words
            words = emailcontent.split()
            filtered_words = [tk for tk in words if tk not in stopwords]

            #we write the ID , current Spam classification, total word count in document
            #the output file will contain one row for every document
            if emaildata[1] == "1":
                outtxt = emaildata[0]+'\t'+ "1" +'\t'+ str(len(filtered_words)) 
            else:
                outtxt = emaildata[0]+'\t'+ "0" +'\t'+ str(len(filtered_words)) 
    
            #Prepare our word count list
            #Add word to dict if in findword list or all words '*' specified
            wc = {}
            for word in filtered_words:
                    wc[word] = wc.get(word, 0) + 1
            
            #Stick the occurence of word=count for every word in document
            #for faster processing on the reducer side.
            if(len(wc) > 0):
                wordcounttxt = "";
                for word, count in wc.iteritems():
                    # *************************************************
                    # We Ignore words with Freq Count < 3
                    # *************************************************
                    if count > 2: 
                        wordcounttxt += '\t'+word+"="+str(count)
            
            #Just capture the case where there are not valid words
            if wordcounttxt == "":
                wordcounttxt = "\t*"
                
            #write out document row with 
            print outtxt + wordcounttxt

Overwriting highfreqwords_mapper.py


In [9]:
#Clean up old data output
!hdfs dfs -rm /user/dsq/HW25output/*
!hdfs dfs -rmdir /user/dsq/HW25output

16/01/26 03:25:01 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
rm: `/user/dsq/HW25output/*': No such file or directory
16/01/26 03:25:03 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [10]:
# !$HADOOP_INSTALL/bin/hadoop jar $HADOOP_INSTALL/hadoop-streaming-2.7.1.jar 
    # -mapper highfreqwords_mapper.py 
    # -reducer reducer.py 
    # -input enronemail_1h.txt 
    # -output HW24output 
    # -file mapper.py 
    # -file reducer.py 
    # -file enronemail_1h.txt 
    # -jobconf mapred.reduce.tasks=1

!$HADOOP_INSTALL/bin/hadoop jar $HADOOP_INSTALL/hadoop-streaming-2.7.1.jar -mapper highfreqwords_mapper.py -reducer reducer.py -input enronemail_1h.txt -output HW25output -file mapper.py -file reducer.py -file enronemail_1h.txt -jobconf mapred.reduce.tasks=1

16/01/26 03:25:28 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
16/01/26 03:25:28 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/26 03:25:29 WARN streaming.StreamJob: -jobconf option is deprecated, please use -D instead.
16/01/26 03:25:29 INFO Configuration.deprecation: mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
packageJobJar: [mapper.py, reducer.py, enronemail_1h.txt] [] /tmp/streamjob8898892588560369024.jar tmpDir=null
		...
		...
	Map-Reduce Framework
		Map input records=100
		Map output records=100
		Map output bytes=14797
		Map output materialized bytes=15036
		Input split bytes=101
		Combine input records=0
		Combine output records=0
		Reduce input groups=100
		Reduce shuffle bytes=15036
		Reduce input records=100
		Reduce output records=100
		Spilled Records=200
		Shuffled Maps =1
		Failed Shuffles=0
		Merged Map ou

In [11]:
!rm HW25_Classification_With_LPSmoothing_And_HighFreq_Words.txt
!hdfs dfs -copyToLocal HW25output/part-00000 HW25_Classification_With_LPSmoothing_And_HighFreq_Words.txt

rm: cannot remove ‘HW25_Classification_With_LPSmoothing_And_HighFreq_Words.txt’: No such file or directory
16/01/26 03:27:07 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/26 03:27:08 WARN hdfs.DFSClient: DFSInputStream has been closed already


#### Run Training Evaluation for 2.5

In [16]:
#conda install pandas 
import pandas as pd 

def eval_2_5():
    with open('HW25_Classification_With_LPSmoothing_And_HighFreq_Words.txt','rb') as f:
        mr_data=pd.read_csv(f, sep='\t', header=None)
    print "HW 2.5 Multinomial NB Results via Hadoop MapReduce Implementation With LP Smoothing With word freq > 2"
    print "Misclassification Error Rate" 
    calculate_training_error(mr_data[1],mr_data[2])

eval_2_5()

HW 2.5 Multinomial NB Results via Hadoop MapReduce Implementation With LP Smoothing With word freq > 2
Misclassification Error Rate
Training error: 0.13


## HW 2.6 Benchmark your code with the Python SciKit-Learn implementation of the multinomial Naive Bayes algorithm

It always a good idea to benchmark your solutions against publicly available libraries such as SciKit-Learn, The Machine Learning toolkit available in Python. In this exercise, we benchmark ourselves against the SciKit-Learn implementation of multinomial Naive Bayes.  For more information on this implementation see: http://scikit-learn.org/stable/modules/naive_bayes.html more  

In this exercise, please complete the following:

* Run the Multinomial Naive Bayes algorithm (using default settings) from SciKit-Learn over the same training data used in HW2.5 and report the misclassification error (please note some data preparation might be needed to get the Multinomial Naive Bayes algorithm from SkiKit-Learn to run over this dataset)
* Prepare a table to present your results, where rows correspond to approach used (SkiKit-Learn versus your Hadoop implementation) and the column presents the training misclassification error
* Explain/justify any differences in terms of training error rates over the dataset in HW2.5 between your Multinomial Naive Bayes implementation (in Map Reduce) versus the Multinomial Naive Bayes implementation in SciKit-Learn

## HW 2.6.1 OPTIONAL (note this exercise is a stretch HW and optional)
* Run the Bernoulli Naive Bayes algorithm from SciKit-Learn (using default settings) over the same training data used in HW2.6 and report the misclassification error 
* Discuss the performance differences in terms of misclassification error rates over the dataset in HW2.5 between the  Multinomial Naive Bayes implementation in SciKit-Learn with the  Bernoulli Naive Bayes implementation in SciKit-Learn. Why such big differences. Explain. 

***NOTE:***
* The following code assumes "enronemail_1h.txt" is in the same folder as this notebook.
* Results from HW2.4 are hard coded into the results table for conveinience.

In [19]:
#!/usr/bin/python
## 
## Author: Tigi Thomas
## Description: Benchmark Classifier using Scikit Learn for HW 1.6
import sys
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB

from IPython.display import display, HTML


#fixed list of stop words
stopwords = ['a','able','about','across','after','all','almost','also','am','among',
             'an','and','any','are','as','at','be','because','been','but','by','can',
             'cannot','could','dear','did','do','does','either','else','ever','every',
             'for','from','get','got','had','has','have','he','her','hers','him','his',
             'how','however','i','if','in','into','is','it','its','just','least','let',
             'like','likely','may','me','might','most','must','my','neither','no','nor',
           'not','of','off','often','on','only','or','other','our','own','rather','said',
             'say','says','she','should','since','so','some','than','that','the','their',
             'them','then','there','these','they','this','tis','to','too','twas','us',
             'wants','was','we','were','what','when','where','which','while','who',
             'whom','why','will','with','would','yet','you','your']

#Use some pre-compiled re-gex for punctuation and numbers
punctpattern = re.compile(r'[\.\^\$\*\+\-\=\{\}\[\]\\\|\(\)<>-@#%_=!:;,/\'\"]')
numpatt1 = re.compile(r'\b[0-9]{2,100}\b')
numpatt2 = re.compile(r'[0-9]{2,100}\b')
numpatt3 = re.compile(r'\b[0-9]{2,100}')
stpwordspattern = re.compile(r'\b(' + r'|'.join(stopwords) + r')\b')

# Preprocess any text
def preprocess_txt(s): 
    s = s.strip().lower()
    s = re.sub(punctpattern, r'',s)
    s = re.sub(numpatt1, r'', s )
    s = re.sub(numpatt2, r'', s )
    s = re.sub(numpatt3, r'', s )
    s = re.sub(stpwordspattern, r'',s)
    return s

email = []
emailclass = []
data = []

# Read Data File and setup Prep Data for Scikit Learn Model
filename = "enronemail_1h.txt"
with open (filename, "rU") as myfile:
    for line in myfile:
        line = line.lower()
        emaildata = line.split("\t")
        emaildatalen = len(emaildata)
        
        #don't really care about the first item, 
        #start from spam or not.. 
        if(emaildatalen >= 2):            
            if(emaildatalen >= 3 ):
                subject = emaildata[2]
            else:
                subject = " "
            
            if(emaildatalen == 4 ):
                body = emaildata[3]
            else:
                body = " "
                    
            emailcontent = subject + " "  + body 
            
            #pre-process to remove punctuation etc.
            emailcontent = preprocess_txt(emailcontent).strip();
            
            data.append([emaildata[0], emailcontent])
            email.append(emailcontent)
            emailclass.append(emaildata[1])       
                      

# Shuffle and create Training and Test data Set- both are same for now.
emails = np.array(data)
X, Y = np.array(email), np.array(emailclass)

np.random.seed(0)
shuffle = np.random.permutation(np.arange(X.shape[0]))
X, Y = X[shuffle], Y[shuffle]
emails = emails[shuffle]

#Both Test and Train are same data set as 
#per instructions in HW.
train_data, train_labels  = X, Y
test_data,  test_labels   = X, Y

##checks
#print emails.shape
#print 'training label shape:', train_labels.shape
#print 'test label shape:', test_labels.shape

#Initialize Count Vectorizer
count_vect = CountVectorizer()
#fit and transform
fitX_train_counts = count_vect.fit_transform(train_data)
fitX_test_counts  = count_vect.fit_transform(test_data)


# Run thru the different Scikit Learn Models and Compare accuracy.

print "Error Rates: Benchmark Comparison between Scikit Learn and HW 2.4 "
print "----------------------------------------------------------------"
results = []

# ----------------------- MultinomialNB --------------------------
mnb = MultinomialNB() #(alpha = a)
mnb.fit(fitX_train_counts, train_labels)

# Predict using our Model
mnb_predicted = mnb.predict(fitX_test_counts)
accuracy  = round(np.where(mnb_predicted == test_labels, 1, 0).sum() / float(len(test_data)),5)
error = round(1 - accuracy,5)
#print 'Error Rate : Scikit-Learn Multinomia NB = {0:.5f}'.format(error)
results.append(['Scikit MultinomialNB', accuracy, error])

# ----------------------- BernoulliNB --------------------------
bnb = BernoulliNB() #(alpha = a)
bnb.fit(fitX_train_counts, train_labels)

# Predict using our Model
bnb_predicted = bnb.predict(fitX_test_counts)
accuracy = round(np.where(bnb_predicted == test_labels, 1, 0).sum() / float(len(test_data)),5)
error = round(1 - accuracy,5)
#print 'Error Rate : Sciki-Learn Bernoulli NB = {0:.5f}'.format(error)
results.append(['Scikit BernoulliNB', accuracy, error])

# ----------------------- HW1.5 --------------------------
#Add in error rate from HW1.5 above
#print 'Error Rate : HW1.5 MulinomilNB = {0:.5f}'.format(0.0)
results.append(['HW 2.4 MulinomilNB', 1.0, 0.0])

# Tabularize the Results
results = pd.DataFrame(results, columns=["Model", "Accuracy", "Error"])
display(results)
#HTML(results.to_html())

Error Rates: Benchmark Comparison between Scikit Learn and HW 2.4 
----------------------------------------------------------------


,Model,Accuracy,Error
0,Scikit MultinomialNB,1.00,0.00
1,Scikit BernoulliNB,0.77,0.23
2,HW 2.4 MulinomilNB,1.00,0.00


### Explain/justify any differences in terms of training error rates over the dataset in HW1.5 between your Multinomial Naive Bayes implementation (in Map Reduce) versus the Multinomial Naive Bayes implementation in SciKit-Learn (Hint: smoothing, which we will discuss in next lecture)

   When using both Scikit MultinomialNB and MultinomialNB fro m HW1.5, the error rates were zero. In both cases the both the entire data set was used to train and to test the model. Additionally, Smoothing was used in HW1.5, and the default Scikit MultinomialNB 'alpha' (Additive Laplace smoothing parameter (0 for no smoothing)) is 1.0 , resulting in similar results.

### Discuss the performance differences in terms of training error rates over the dataset in HW1.5 between the  Multinomial Naive Bayes implementation in SciKit-Learn with the  Bernoulli Naive Bayes implementation in SciKit-Learn

   BernoulliNB implements the naive Bayes assuming each feature to be a binary-valued (Bernoulli, boolean) variable. Therefore samples are required to be represented as binary-valued feature vectors. The decision rule for Bernoulli NB explicitly penalizes non-occurrence of a feature versus MultinomialNB which ignores a non-occurring feature. Since the feature vectors are now yes or no vectors versus count vectors, with larger documents there is more chance of error.